In [7]:
import spacy
import nltk
from nltk.tokenize import word_tokenize
import os
import csv
import re
import string
import pandas as pd


In [57]:
def recognite(data_path, nlp, ner_categories):
    entities = []
    for sub_dir in os.listdir(data_path):
        for f in os.listdir(os.path.join(data_path, sub_dir)):
            with open(os.path.join(data_path, sub_dir, f), 'r') as file:
                text = file.read()
                for ent in nlp(text).ents:
                    if ent.label_ in ner_categories:
                        entities.append(ent.text)
    return entities
    


In [62]:
data_path = '../data/txt/ouvrages_mixtes'
nlp = spacy.load('fr_core_web_sm')
ner_categories =['PER']

doc = recognite(data_path, nlp, ner_categories)

In [63]:
def extract_names(data_path):
    for subdirectory in os.listdir(data_path):
        subdirectory_path = os.path.join(data_path, subdirectory)
        if os.path.isdir(subdirectory_path):

            num_pages = len([filename for filename in os.listdir(subdirectory_path) if filename.endswith('.txt')])

            for filename in os.listdir(subdirectory_path):
                if filename.endswith('.txt'):
                    file_path = os.path.join(subdirectory_path, filename)
                    with open(file_path, 'r', encoding='utf-8') as file:
                        content = file.read()
                    
                        title = file_path.split('/')[4]
                        
                        author_pattern = re.compile(r'\b(Mme|Madame|M|Mr|Dr|Docteur|docteur|Monsieur|monsieur|madame|professeur|Professeur)\b\s+(\S+\s+\S+\s+\S+)|(?<!\.\s)(\b[A-Z][a-z]+\b)')
                        author_match = author_pattern.search(content[:500])
                        author = author_match.group(0) if author_match else 'auteur inconnu'
                        data_mixtes = []
                        if author != 'auteur inconnu':
                            data_mixtes.append([title, author, num_pages])

    df = pd.DataFrame(data_mixtes, columns=['Titre', 'Auteur', 'Nbr_pages'])
    context_name = data_path.split('_')[-1]
    df.to_csv(f'../data/contexte/contexte_{context_name}.csv', index=False)
    return title, author

[]